In [104]:
import unicodecsv
enrollments_filename='../data/enrollments.csv'
engagements_filename='../data/daily_engagement.csv'
submissions_filename='../data/project_submissions.csv'

In [105]:
def open_file(filename):
    with open(filename,'rb') as f:
        reader=unicodecsv.DictReader(f)
        text=list(reader)
    return text

In [106]:
enrollments=open_file(enrollments_filename)
daily_engagement=open_file(engagements_filename)
project_submissions=open_file(submissions_filename)

In [107]:
len(enrollments)

unique_enrolled_students = set()
for enrollment in enrollments:
    unique_enrolled_students.add(enrollment['account_key'])
print 'unique enrolled student',len(unique_enrolled_students)

len(daily_engagement)

unique_engagement_students = set()
for engagement_record in daily_engagement:
    unique_engagement_students.add(engagement_record['acct'])
print 'unique engagement student',len(unique_engagement_students)

len(project_submissions)

unique_project_submitters = set()
for submission in project_submissions:
    unique_project_submitters.add(submission['account_key'])
print 'unique project submitters ',len(unique_project_submitters)

unique enrolled student 1302
unique engagement student 1237
unique project submitters  743


In [108]:
from datetime import datetime as dt
def parse_date(date):
    if date=='':
        return None
    else:
        return dt.strptime(date,'%Y-%m-%d')
def parse_maybe_int(i):
    if i=="":
        return None
    else:
        return int(i)


In [109]:
for enrollment in enrollments:
    enrollment['cancel_date']=parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel']=parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled']=enrollment['is_canceled']=='True'
    enrollment['is_udacity']=enrollment['is_udacity']=='True'
    enrollment['join_date']=parse_date(enrollment['join_date'])
    

for engagement_record in daily_engagement:
    engagement_record['lessons_completed']=int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited']=int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed']=int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited']=float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date']=parse_date(engagement_record['utc_date'])

for submission in project_submissions:
    submission['completion_date']=parse_date(submission['completion_date'])
    submission['creation_date']=parse_date(submission['creation_date'])

In [110]:
for engagement_record in daily_engagement:
    engagement_record['account_key'] = engagement_record['acct']
    del[engagement_record['acct']]

In [111]:
def get_unique_students(data):
    unique_students = set()
    for data_point in data:
        unique_students.add(data_point['account_key'])
    return unique_students
len(enrollments)
unique_enrolled_students = get_unique_students(enrollments)
len(unique_enrolled_students)
len(daily_engagement)
unique_engagement_students = get_unique_students(daily_engagement)
len(unique_engagement_students)
len(project_submissions)
unique_project_submitters = get_unique_students(project_submissions)
len(unique_project_submitters)

743

In [112]:
print daily_engagement[0]['account_key']

0


In [113]:
for enrollment in enrollments:
    student = enrollment['account_key']
    if student not in unique_engagement_students:
        print enrollment,"    ",student
        break

{u'status': u'canceled', u'is_udacity': False, u'is_canceled': True, u'join_date': datetime.datetime(2014, 11, 12, 0, 0), u'account_key': u'1219', u'cancel_date': datetime.datetime(2014, 11, 12, 0, 0), u'days_to_cancel': 0}      1219


In [114]:
num_problem_students = 0
for enrollment in enrollments:
    student = enrollment['account_key']
    if (student not in unique_engagement_students and 
            enrollment['join_date'] != enrollment['cancel_date']):
        print enrollment
        num_problem_students += 1

num_problem_students

{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2015, 1, 10, 0, 0), u'account_key': u'1304', u'cancel_date': datetime.datetime(2015, 3, 10, 0, 0), u'days_to_cancel': 59}
{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2015, 3, 10, 0, 0), u'account_key': u'1304', u'cancel_date': datetime.datetime(2015, 6, 17, 0, 0), u'days_to_cancel': 99}
{u'status': u'current', u'is_udacity': True, u'is_canceled': False, u'join_date': datetime.datetime(2015, 2, 25, 0, 0), u'account_key': u'1101', u'cancel_date': None, u'days_to_cancel': None}


3

In [115]:
udacity_test_accounts=set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])
len(udacity_test_accounts)
#print udacity_test_accounts

6

In [116]:
def remove_udacity_accounts(data):
    non_udacity_data=[]
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
    return non_udacity_data

In [117]:
non_udacity_enrollments=remove_udacity_accounts(enrollments)
non_udacity_engagement=remove_udacity_accounts(daily_engagement)
non_udacity_submissions=remove_udacity_accounts(project_submissions)

print len(non_udacity_enrollments)
print len(non_udacity_engagement)
print len(non_udacity_submissions)

1622
135656
3634


In [118]:
paid_students = {}
for enrollment in non_udacity_enrollments:
    if (not enrollment['is_canceled'] or
            enrollment['days_to_cancel'] > 7):
        account_key = enrollment['account_key']
        enrollment_date = enrollment['join_date']
        if (account_key not in paid_students or
                enrollment_date > paid_students[account_key]):
            paid_students[account_key] = enrollment_date
len(paid_students)

995

In [121]:
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days < 7

def remove_free_trial_cancels(data):
    new_data = []
    for data_point in data:
        if data_point['account_key'] in paid_students:
            new_data.append(data_point)
    return new_data

paid_enrollments = remove_free_trial_cancels(non_udacity_enrollments)
paid_engagement = remove_free_trial_cancels(non_udacity_engagement)
paid_submissions = remove_free_trial_cancels(non_udacity_submissions)

print len(paid_enrollments)
print len(paid_engagement)
print len(paid_submissions)

paid_engagement_in_first_week = []
for engagement_record in paid_engagement:
    account_key = engagement_record['account_key']
    join_date = paid_students[account_key]
    engagement_record_date = engagement_record['utc_date']

    if within_one_week(join_date, engagement_record_date):
        paid_engagement_in_first_week.append(engagement_record)

len(paid_engagement_in_first_week)

1293
134549
3618


21508